In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle").sort_values(["user_id", "timestamp"]).reset_index(drop=True)

In [3]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [15]:
for i in range(4):
    df[f"ans{i}_size"] = df["user_answer"] == i
    df[f"ans{i}_size"] = df.groupby("user_id")[f"ans{i}_size"].transform("sum") / df.groupby("user_id")["user_id"].transform("count")

In [16]:
df[df["ans0_size"] == 1]["user_id"].value_counts()

837177950     2
1231833900    1
719804730     1
1527175400    1
1777952220    1
1632816630    1
2119645200    1
Name: user_id, dtype: int64

In [17]:
df[df["ans1_size"] == 1]["user_id"].value_counts()

2097498310    89
2100053240    30
1821791870    30
1255907180    30
1031172430    30
2098274630    16
1317856170     7
1880240        1
Name: user_id, dtype: int64

In [18]:
df[df["ans2_size"] == 1]["user_id"].value_counts()

2103184240    1
1240997940    1
Name: user_id, dtype: int64

In [19]:
df[df["ans3_size"] == 1]["user_id"].value_counts()

1610758750    1
Name: user_id, dtype: int64

In [20]:
a = df.groupby("user_id").rolling(window=30).cumstd()

AttributeError: 'RollingGroupby' object has no attribute 'cumstd'

In [23]:
df["timedelta_cap100k"] = [x if x < 100000 else 100000 for x in df["timedelta"].values]
df["mean_timedelta_cap100k"] = df.groupby(["user_id", "part"])["timedelta"].cumsum() / (df.groupby("user_id").cumcount()+1)
df["diff_mean_timedelta_cap100k"] = df["timedelta"] - df["mean_timedelta_cap100k"]

In [24]:
df["diff_mean_timedelta_cap100k"]

0                    NaN
1           12689.500000
2            8045.666667
3           15033.000000
4           62736.000000
                ...     
10033861   -42395.869565
10033862   -54327.829787
10033863   -48124.895833
10033864   -45725.285714
10033865   -55958.280000
Name: diff_mean_timedelta_cap100k, Length: 10033866, dtype: float64

In [26]:
df["elapsed_time"] = df.groupby("user_id")["prior_question_elapsed_time"].shift(-1)
df["elapsed_mean_contant_id"] = df.groupby("content_id")["elapsed_time"].transform("mean")
df["diff_timedelta_elapsed_mean_content_id"] = df["timedelta"] - df["elapsed_mean_contant_id"]

In [32]:
df["study_time"] = (df["timedelta_cap100k"] - df["prior_question_had_explanation"]).astype("float32")
df["diff_mean_study_time"] = df["timedelta_cap100k"] - df.groupby(["user_id", "part"])["study_time"].transform("mean")

In [33]:
df["timedelta_cap100k"] = [x if x < 100000 else 100000 for x in df["timedelta"].values]
correct_dict = df[df["answered_correctly"]==1].groupby(["content_id"])["timedelta_cap100k"].mean().to_dict()
incorrect_dict = df[df["answered_correctly"]==0].groupby(["content_id"])["timedelta_cap100k"].mean().to_dict()

df["diff_correct_timedelta_cap100k"] = df["timedelta_cap100k"] - df["content_id"].map(correct_dict)
df["diff_incorrect_timedelta_cap100k"] = df["timedelta_cap100k"] - df["content_id"].map(incorrect_dict)

In [35]:
df[["timedelta_cap100k", "diff_correct_timedelta_cap100k", "diff_incorrect_timedelta_cap100k"]]

,timedelta_cap100k,diff_correct_timedelta_cap100k,diff_incorrect_timedelta_cap100k
0,100000.0,9994.460586,8364.047148
1,25379.0,-6520.752666,-6294.014079
2,24758.0,-9617.424628,-8597.855108
3,20044.0,-6272.117127,-8355.064081
4,78420.0,53695.235745,54678.881050
...,...,...,...
10033861,34075.0,-9861.153846,-16837.163636
10033862,20962.0,-32408.263274,-35373.680000
10033863,26141.0,-22469.677180,-27384.735294
10033864,27588.0,-20036.299118,-32667.449569
